<a href="https://colab.research.google.com/github/VishalPrem1994/AIGenPlayGround/blob/main/TodayInHistoryGen/StaticImageGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import openai
import datetime
import json
import string
import cv2
import numpy as np
import glob
import os
import random

In [ ]:
class Image:
    def __init__(self, filename, time=500, size=500):
        self.size = size
        self.time = time
        self.shifted = 0.0
        self.img = cv2.imread(filename)
        self.height, self.width, _ = self.img.shape
        if self.width < self.height:
            self.height = int(self.height*size/self.width)
            self.width = size
            self.img = cv2.resize(self.img, (self.width, self.height))
            self.shift = self.height - size
            self.shift_height = True
        else:
            self.width = int(self.width*size/self.height)
            self.height = size
            self.shift = self.width - size
            self.img = cv2.resize(self.img, (self.width, self.height))
            self.shift_height = False
        self.delta_shift = self.shift/self.time
    def reset(self):
        if random.randint(0, 1) == 0:
            self.shifted = 0.0
            self.delta_shift = abs(self.delta_shift)
        else:
            self.shifted = self.shift
            self.delta_shift = -abs(self.delta_shift)
    def get_frame(self):
        if self.shift_height:
            roi = self.img[int(self.shifted):int(self.shifted) + self.size, :, :]
        else:
            roi = self.img[:, int(self.shifted):int(self.shifted) + self.size, :]
        self.shifted += self.delta_shift
        if self.shifted > self.shift:
            self.shifted = self.shift
        if self.shifted < 0:
            self.shifted = 0
        return roi

In [ ]:
def promptsForDate(date,promptType):
    if promptType=="IN":
        return "Summarize two famous historical events that happened on "+str(date.strftime("%d"))+" "+str(date.strftime("%B"))+" In India In history"
    else:
        return "Summarize two famous event that happened on "+str(date.strftime("%d"))+" "+str(date.strftime("%B"))

def promptForImageGeneration(rawText):
    return "Generate three detailed Prompts for giving ai art generation models for the following text "+str(rawText)

def promptsListCleanUp(rawTextList):
    newList = list()
    for i in rawTextList:
        if(len(i)!=0):
            i.translate(str.maketrans('', '', string.punctuation))
            newList.append(i)
    return newList

def getDaVinciResponse(prompt):
    promptResponse = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0.7,
        max_tokens=200,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=1
    )
    promptResponse = promptResponse['choices'][0]['text'].split("\n\n")
    promptResponse = promptsListCleanUp(promptResponse)
    return promptResponse


In [ ]:
def process(path):
    filenames = glob.glob(os.path.join(path, "*"))
    cnt = 0
    images = []
    for filename in filenames:
        print(filename)
        img = Image(filename)
        images.append(img)
        if cnt > 300:
            break
        cnt += 1
    prev_image = images[random.randrange(0, len(images))]
    prev_image.reset()
    while True:
        while True:
            img = images[random.randrange(0, len(images))]
            if img != prev_image:
                break
        img.reset()
        for i in range(100):
            alpha = i/100
            beta = 1.0 - alpha
            dst = cv2.addWeighted(img.get_frame(), alpha, prev_image.get_frame(), beta, 0.0)
            cv2.imshow("Slide", dst)
            if cv2.waitKey(1) == ord('q'):
                return
        prev_image = img
        for _ in range(300):
            cv2.imshow("Slide", img.get_frame())
            if cv2.waitKey(1) == ord('q'):
                return